##### First attach a default lakehouse to this notebook, and then start by loading required libraries below

In [33]:
import requests, json, base64, re
from notebookutils import mssparkutils

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 35, Finished, Available, Finished)

##### Either get user credentials

In [58]:
token = mssparkutils.credentials.getToken('pbi')


StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 60, Finished, Available, Finished)

##### Or use service principal credentials

In [ ]:
# retreive sp details from keyvault
nameOfKeyVault = ''
keyvault = f'https://{nameOfKeyVault}.vault.azure.net/'
tenant_id = mssparkutils.credentials.getSecret(keyvault,tenantId_SecretName)
client_id = mssparkutils.credentials.getSecret(keyvault,clientId_SecretName)
client_secret = mssparkutils.credentials.getSecret(keyvault,clientSecret_SecretName)

# oauth2 login
url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

# Login and get token
payload = f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}}&resource=https://api.fabric.microsoft.com'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
}

response = json.loads(requests.request("POST", url, headers=headers, data=payload).content)
print(response)
# Set the access token
token = response['access_token']

##### Request header, always run this before running the cells below

In [59]:
bearerToken = token; # replace this token with the real AAD token

headers = {
    "Authorization": f"Bearer {bearerToken}", 
    "Content-Type": "application/json"  # Set the content type based on your request
}

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 61, Finished, Available, Finished)

#### Example of creating a spark job definition programatically

See here for more details https://learn.microsoft.com/en-us/fabric/data-engineering/spark-job-definition-api

##### First create a basic spark job definition file in the default lakehouse

In [18]:
sjd_contents = """import sys
import os
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

if __name__ == "__main__":

    #Spark session builder
    spark_session = (SparkSession
          .builder
          .appName("sjdsampleapp") 
          .config("spark.some.config.option", "some-value")
          .getOrCreate())
    
    spark_context = spark_session.sparkContext
    spark_context.setLogLevel("DEBUG")

    print("hello world!") """
mssparkutils.fs.put("Files/sjd_hello_world.py", sjd_contents, True) # Set the last parameter as True to overwrite the file if it existed already


StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 20, Finished, Available, Finished)

True

##### Validate the file was created successfully

In [52]:
file_path = mssparkutils.fs.ls("Files/sjd_hello_world.py")[0].path
mssparkutils.fs.head(file_path, 10000)

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 54, Finished, Available, Finished)

'import sys\nimport os\nfrom pyspark.sql import SparkSession\nfrom pyspark.conf import SparkConf\n\nif __name__ == "__main__":\n\n    #Spark session builder\n    spark_session = (SparkSession\n          .builder\n          .appName("sjdsampleapp") \n          .config("spark.some.config.option", "some-value")\n          .getOrCreate())\n    \n    spark_context = spark_session.sparkContext\n    spark_context.setLogLevel("DEBUG")\n\n    print("hello world!") '

In [54]:
def base64_to_json(base64_data):
    # Decode the Base64-encoded string to bytes
    base64_bytes = base64_data.encode('utf-8')
    
    # Decode the bytes to a JSON string
    json_string = base64.b64decode(base64_bytes).decode('utf-8')
    
    # Deserialize the JSON string to a Python dictionary
    json_data = json.loads(json_string)
    
    return json_data


    
def json_to_base64(json_data):
    # Serialize the JSON data to a string
    json_string = json.dumps(json_data)
    
    # Encode the JSON string as bytes
    json_bytes = json_string.encode('utf-8')
    
    # Encode the bytes as Base64
    base64_encoded = base64.b64encode(json_bytes).decode('utf-8')
    
    return base64_encoded

defaultLakehouseArtifactId = mssparkutils.runtime.context['defaultLakehouseId']

payload_json = {
"executableFile": file_path,
"defaultLakehouseArtifactId": defaultLakehouseArtifactId,
"mainClass":"",
"additionalLakehouseIds":[],
"retryPolicy":"",
"commandLineArguments":"",
"additionalLibraryUris":[],
"language":"",
"environmentArtifactId":""
}

payload = json_to_base64(payload_json)
#payload = "eyJleGVjdXRhYmxlRmlsZSI6bnVsbCwiZGVmYXVsdExha2Vob3VzZUFydGlmYWN0SWQiOiIiLCJtYWluQ2xhc3MiOiIiLCJhZGRpdGlvbmFsTGFrZWhvdXNlSWRzIjpbXSwicmV0cnlQb2xpY3kiOm51bGwsImNvbW1hbmRMaW5lQXJndW1lbnRzIjoiIiwiYWRkaXRpb25hbExpYnJhcnlVcmlzIjpbXSwibGFuZ3VhZ2UiOiIiLCJlbnZpcm9ubWVudEFydGlmYWN0SWQiOm51bGx9"
print(payload)


StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 56, Finished, Available, Finished)

eyJleGVjdXRhYmxlRmlsZSI6ICJhYmZzczovLzcwOTA5Y2EyLWNlYmMtNGNiOC05MGY0LWNkZWM3ODJkZGVlZUBvbmVsYWtlLmRmcy5mYWJyaWMubWljcm9zb2Z0LmNvbS9mZjYzZmE1Ny1jZGQyLTQzYjMtYWMyOC01MTJlYWYyYWYzNmMvRmlsZXMvc2pkX2hlbGxvX3dvcmxkLnB5IiwgImRlZmF1bHRMYWtlaG91c2VBcnRpZmFjdElkIjogImZmNjNmYTU3LWNkZDItNDNiMy1hYzI4LTUxMmVhZjJhZjM2YyIsICJtYWluQ2xhc3MiOiAiIiwgImFkZGl0aW9uYWxMYWtlaG91c2VJZHMiOiBbXSwgInJldHJ5UG9saWN5IjogIiIsICJjb21tYW5kTGluZUFyZ3VtZW50cyI6ICIiLCAiYWRkaXRpb25hbExpYnJhcnlVcmlzIjogW10sICJsYW5ndWFnZSI6ICIiLCAiZW52aXJvbm1lbnRBcnRpZmFjdElkIjogIiJ9


In [60]:
#####################################################################################
### Either use current workspace ID or set your workspace ID and desired job name ###
#####################################################################################
workspace_id = mssparkutils.runtime.context['currentWorkspaceId'] 
#workspace_id = '7be392e2-d65b-45f0-83f4-7d82ffb36cc0'
job_name = "SJDHelloWorld9"


url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items"
# Define the payload data for the POST request
payload_data = {
    "displayName": job_name,
    "Type": "SparkJobDefinition",
    "definition": {
        "format": "SparkJobDefinitionV1",
        "parts": [
            {
                "path": "SparkJobDefinitionV1.json",
                "payload": payload,
                "payloadType": "InlineBase64"
            }
        ]
    }
}

# Make the POST request with Bearer authentication
response = requests.post(url, json=payload_data, headers=headers)
print(response.text)
job_definition_id = (response.json()['id'])
print('\nJob definition id is '+job_definition_id)

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 62, Finished, Available, Finished)

{"id":"656ecc32-8bdc-4517-8edb-ff838c534b2d","type":"SparkJobDefinition","displayName":"SJDHelloWorld9","description":"","workspaceId":"7be392e2-d65b-45f0-83f4-7d82ffb36cc0"}

Job definition id is 656ecc32-8bdc-4517-8edb-ff838c534b2d


##### Confirm job creation - Fetches all the spark job definitions in a particular workspace

In [61]:
headers = {
    "Authorization": f"Bearer {bearerToken}", 
    "Content-Type": "application/json"  # Set the content type based on your request
}
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items?type=SparkJobDefinition"
response = requests.get(url, headers=headers)
display(spark.read.json(sc.parallelize(response.json()['value'])))

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 63, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dd9c3df9-2ed7-4449-a94e-091aa43e5da6)

##### Invoke a Spark job created above

In [62]:
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{job_definition_id}/jobs/instances?jobType=sparkjob"
payload_data = {}
response = requests.post(url, json=payload_data, headers=headers)
sjd_instance_id = response.headers['Location'].split('/')[10]
print("Job invoked, returned instance ID = "+sjd_instance_id)

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 64, Finished, Available, Finished)

Job invoked, returned instance ID = 2f12efac-d351-427a-958a-31c36edd0fda


##### Get spark job definition status

In [63]:
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{job_definition_id}/jobs/instances/{sjd_instance_id}"
response = requests.get(url, json=payload_data, headers=headers)
print(response.text)
sjd_status = response.json()['status']
print('\nJob status is: '+sjd_status)

StatementMeta(, 32c765f8-674a-4120-bac9-e996206e811a, 65, Finished, Available, Finished)

{"id":"2f12efac-d351-427a-958a-31c36edd0fda","itemId":"656ecc32-8bdc-4517-8edb-ff838c534b2d","jobType":"sparkjob","invokeType":"Manual","status":"InProgress","failureReason":null,"rootActivityId":"8308ae7f-eca7-4ba4-bca1-72a9eeb44ade","startTimeUtc":"2024-07-25T20:00:09.7966667","endTimeUtc":null}

Job status is: InProgress
